<a href="https://colab.research.google.com/github/markjschmidt/PET_List/blob/main/My_OpenAI_04_mini_List_file_Headers_20250515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is a follow up to simply reviewing the readlist.m file after discovering it relies on additional MATLAB files for reading header data in our Vereos List files.

The supported List mode acquisition file type is 'XTAL' 16, the 8-byte Crystal List Mode that has no timestamps between buffers.

It has two 32-bit data words of XPOS(B)-XPOS(A) ring X direction crystal followed by Z(B)-Z(A) Axial direction crystal numbers.

This is followed by 32-bit control words that can cotain Singles Rate, Prompt Rate, Delays Rate, Elapsed time from Acq Start or Table Horizontal position values.







In [ ]:
!pwd

/content


In [ ]:
!ls -la


total 1620
drwxr-xr-x 1 root root    4096 May 16 03:57 .
drwxr-xr-x 1 root root    4096 May 16 03:54 ..
-rw-r--r-- 1 root root 1641744 May 16 03:57 3OCLK_2bd6.list
drwxr-xr-x 4 root root    4096 May 14 13:38 .config
drwxr-xr-x 1 root root    4096 May 14 13:38 sample_data


## Focus on printing important Header values that are needed to help interpret the event data.

My Prompt:

I have attached additional MATLAB files to help with this follow up question. Using python in Google Colab, how can I print the main header (see imageio_getmhdr.m), the extended main header (see imageio_getexhdr.m) and the sub header (see imageio_getshdr.m) for a .list file? I have attached a relatively small .list file (see 3OCLK_2bd6.list) to analyze.


o4:

Reading documents

The user uploaded multiple MATLAB files and a .list file, requesting Python code to replicate the functionality of three MATLAB functions. Their goal is to use Python, specifically in Google Colab, to parse and print the main header, extended main header, and sub-header in a similar fashion to the MATLAB scripts. I’ll need to write a Python script using the struct module to read and unpack the binary data from the .list file at the specified offsets. The user prefers not using MATLAB code here.

To replicate the MATLAB code in Python, I need to:

Open the file in 'rb' mode (big-endian).

For the main header: seek to offset 512 and read 27 uint16s, plus various fields like file_fmt and scan_geom. I can use a class or dictionary to store these fields.

For the extended header: seek to offset 1024 and read data after the main header without repositioning the file pointer. This mimics the MATLAB imageio_getexhdr behavior.

For the sub-header: after the extended header, seek to 9216 to get to the sub-header.

I’ll implement this sequential logic in Python.

To replicate the MATLAB logic in Python, I can use fixed offsets for each header. Start by seeking to offset 512 for the main header, reading 512 bytes, and using struct to unpack the data (27 uint16 values, a uint32, float32 values, etc.). After that, move to offset 1024 for the extended header and read further bytes using struct.unpack. Finally, for the subheader, seek to 9216 and follow a similar process. I could write a general function to read these structured fields into a Python dictionary for clarity and ease of use.



In [ ]:
import struct
import pprint

def _read_uint16(f, count=1):
    """Read big-endian unsigned 16-bit integers."""
    data = f.read(2 * count)
    return struct.unpack('>' + 'H' * count, data) if count > 1 else struct.unpack('>H', data)[0]

def _read_uint32(f, count=1):
    """Read big-endian unsigned 32-bit integers."""
    data = f.read(4 * count)
    return struct.unpack('>' + 'I' * count, data) if count > 1 else struct.unpack('>I', data)[0]

def _read_float32(f, count=1):
    """Read big-endian IEEE 754 floats."""
    data = f.read(4 * count)
    return struct.unpack('>' + 'f' * count, data) if count > 1 else struct.unpack('>f', data)[0]

def read_main_header(filepath):
    hdr = {}
    with open(filepath, 'rb') as f:
        # Main header starts at byte offset 512
        f.seek(512)
        # Signature and core fields: 27 x uint16 (bytes 0–53 relative)
        vals = struct.unpack('>27H', f.read(54))
        # Check characteristic signature (256,0,22)
        sig = (vals[0], vals[1], vals[2])
        if sig != (256, 0, 22):
            hdr['signature_warning'] = sig
        hdr['file_fmt']   = vals[3]
        hdr['scan_geom'] = vals[4]
        hdr['hw_config'] = vals[5]
        hdr['edit_flag'] = vals[8]
        if hdr['file_fmt'] > 13:
            hdr['list_byte_ord'] = vals[9]
        hdr['filtyp']    = vals[26]

        # Next: 1 x uint32 (minTransXtalDiff), 1 x float32 (tofTstampScale)
        hdr['minTransXtalDiff'] = _read_uint32(f)
        hdr['tofTstampScale']   = _read_float32(f)

        # Next 12 x uint16: creation date/time + acquisition flags
        date_vals = struct.unpack('>12H', f.read(24))
        hdr.update({
            'daycre':  date_vals[2],
            'mocre':   date_vals[3],
            'yrcre':   date_vals[4],
            'hrcre':   date_vals[5],
            'mincre':  date_vals[6],
            'seccre':  date_vals[7],
            'duratn':  date_vals[8],
            'shdtyp':  date_vals[9],
            'sngpscl': date_vals[10],
            'singopt': date_vals[11]
        })

        # Next floats: pscale, detectorRadius
        hdr['pscale']         = _read_float32(f)
        hdr['detectorRadius'] = _read_float32(f)

        # Next 5 x uint16: virtualXtal, phiMashing, polygonSides, xtalsPerSide, nXtalRows
        v = struct.unpack('>5H', f.read(10))
        hdr.update(dict(zip(
            ['virtualXtal','phiMashing','polygonSides','xtalsPerSide','nXtalRows'], v
        )))

        # Next floats: crystalThickness, xXtalPitch, zXtalPitch, axialFov
        floats = struct.unpack('>4f', f.read(16))
        hdr.update(dict(zip(
            ['crystalThickness','xXtalPitch','zXtalPitch','axialFov'], floats
        )))

        # Next 11 x uint16: rphiType, sliceType, delayType, pattyp, scntyp, numray, numang, slcthk, (plus isotop if file<15)
        uints = struct.unpack('>11H', f.read(22))
        hdr.update({
            'rphiType':  uints[0],
            'sliceType': uints[1],
            'delayType': uints[2],
            'pattyp':    uints[5],
            'scntyp':    uints[6],
            'numray':    uints[7],
            'numang':    uints[8],
            'slcthk':    uints[9]
        })
        if hdr['file_fmt'] < 15:
            hdr['isotop'] = uints[10]
    return hdr

def read_extended_header(filepath, file_fmt):
    hdr = {}
    with open(filepath, 'rb') as f:
        # Extended header starts at byte offset 1024
        f.seek(1024)
        # Bytes 0–255: 4 strings of 64 chars
        names = ['Dpat_name','Dpat_id','study_uid','series_uid']
        for name in names:
            hdr[name] = f.read(64).decode('ascii').rstrip('\x00')
        # Bytes 256–295: 40 bytes: view_code (20), sortproto_name (20 if fmt<15)
        hdr['view_code'] = f.read(20).decode('ascii').rstrip('\x00')
        if file_fmt < 15:
            hdr['sortproto_name'] = f.read(20).decode('ascii').rstrip('\x00')
        else:
            f.seek(20, 1)
        # Bytes 296–297: uint16 route
        hdr['route'] = _read_uint16(f)
        # Bytes 298–299: pharm (if fmt<15) else skip
        if file_fmt < 15:
            hdr['pharm'] = _read_uint16(f)
        else:
            f.seek(2, 1)
        # Bytes 300–363: req_phys (64 chars)
        hdr['req_phys'] = f.read(64).decode('ascii').rstrip('\x00')
        # Next uint16: card_phstate
        hdr['card_phstate'] = _read_uint16(f)
        # Next int32: assay_date; next uint32: assay_time
        hdr['assay_date'] = struct.unpack('>i', f.read(4))[0]
        hdr['assay_time'] = _read_uint32(f)
        # Next 64 chars: series_desc
        hdr['series_desc'] = f.read(64).decode('ascii').rstrip('\x00')
        # You can continue reading other fields per imageio_getexhdr.m...
    return hdr

def read_sub_header(filepath, file_fmt):
    hdr = {}
    with open(filepath, 'rb') as f:
        # Sub-header starts at byte offset 18*512 = 9216
        f.seek(9216)
        # Bytes 0–15: 8 x uint16; version is 8th
        vals = struct.unpack('>8H', f.read(16))
        hdr['version'] = vals[7]
        # Bytes 16–31: 16 chars: atten_corr
        hdr['atten_corr'] = f.read(16).decode('ascii').rstrip('\x00')
        # Bytes 32–47: 4 floats: actual_bedpos + 3 orientation floats
        bed_and_orient = struct.unpack('>4f', f.read(16))
        hdr['actual_bedpos'] = bed_and_orient[0]
        hdr['orientation']   = bed_and_orient[1:]
        # Bytes 52–63: 3 floats: orientation[3:6]
        more_orient = struct.unpack('>3f', f.read(12))
        hdr['orientation'] += more_orient
        # Bytes 64–79: 4 x uint32: card_fr_time, card_high_rr, card_low_rr, card_tr_time
        cft = struct.unpack('>4I', f.read(16))
        hdr.update(dict(zip(
            ['card_fr_time','card_high_rr','card_low_rr','card_tr_time'], cft
        )))
        # Continue following imageio_getshdr.m for other fields...
    return hdr

# Usage example (in Colab):
filepath = '3OCLK_2bd6.list'
main_hdr = read_main_header(filepath)
ex_hdr   = read_extended_header(filepath, main_hdr['file_fmt'])
sub_hdr  = read_sub_header(filepath, main_hdr['file_fmt'])

print("=== Main Header ===")
pprint.pprint(main_hdr)
print("\n=== Extended Header ===")
pprint.pprint(ex_hdr)
print("\n=== Sub Header ===")
pprint.pprint(sub_hdr)



=== Main Header ===
{'axialFov': 164.0,
 'crystalThickness': 19.0,
 'daycre': 31,
 'delayType': 1,
 'detectorRadius': 382.0,
 'duratn': 0,
 'edit_flag': 0,
 'file_fmt': 15,
 'filtyp': 6,
 'hrcre': 18,
 'hw_config': 3697,
 'list_byte_ord': 1,
 'minTransXtalDiff': 84,
 'mincre': 0,
 'mocre': 11,
 'nXtalRows': 40,
 'numang': 65535,
 'numray': 65535,
 'pattyp': 1,
 'phiMashing': 65535,
 'polygonSides': 18,
 'pscale': 1.0,
 'rphiType': 1,
 'scan_geom': 113,
 'scntyp': 3,
 'seccre': 0,
 'shdtyp': 1,
 'singopt': 0,
 'slcthk': 1,
 'sliceType': 2,
 'sngpscl': 0,
 'tofTstampScale': 1.9531249881299395e-11,
 'virtualXtal': 0,
 'xXtalPitch': 4.099999904632568,
 'xtalsPerSide': 32,
 'yrcre': 1969,
 'zXtalPitch': 4.099999904632568}

=== Extended Header ===
{'Dpat_id': '00000087',
 'Dpat_name': 'SERVICE_Anonymous^BUTTON SRC EMISSION',
 'assay_date': 0,
 'assay_time': 0,
 'card_phstate': 0,
 'req_phys': '',
 'route': 1,
 'series_desc': '9O Emission QC',
 'series_uid': '1.3.46.670589.28.2.2.440632742027

o4:

The code above defines three functions:

read_main_header — seeks to byte 512 in the list file, reads the same fields that imageio_getmhdr.m does (signature, format version, geometry, timing scales, etc.), and returns them in a Python dict.

read_extended_header — seeks to byte 1024, reads patient and study IDs, view codes, measurement timestamps, etc., exactly as imageio_getexhdr.m lays out.

read_sub_header — seeks to byte 9216, reads gating/version information, table position, image orientation, etc., matching imageio_getshdr.m.

You can drop this into a Colab cell (just upload 3OCLK_2bd6.list alongside these functions) and run:


In [ ]:
main_hdr = read_main_header('3OCLK_2bd6.list')
ex_hdr   = read_extended_header('3OCLK_2bd6.list', main_hdr['file_fmt'])
sub_hdr  = read_sub_header('3OCLK_2bd6.list', main_hdr['file_fmt'])

import pprint
print("=== Main Header ===");      pprint.pprint(main_hdr)
print("\n=== Extended Header ==="); pprint.pprint(ex_hdr)
print("\n=== Sub Header ===");     pprint.pprint(sub_hdr)



=== Main Header ===
{'axialFov': 164.0,
 'crystalThickness': 19.0,
 'daycre': 31,
 'delayType': 1,
 'detectorRadius': 382.0,
 'duratn': 0,
 'edit_flag': 0,
 'file_fmt': 15,
 'filtyp': 6,
 'hrcre': 18,
 'hw_config': 3697,
 'list_byte_ord': 1,
 'minTransXtalDiff': 84,
 'mincre': 0,
 'mocre': 11,
 'nXtalRows': 40,
 'numang': 65535,
 'numray': 65535,
 'pattyp': 1,
 'phiMashing': 65535,
 'polygonSides': 18,
 'pscale': 1.0,
 'rphiType': 1,
 'scan_geom': 113,
 'scntyp': 3,
 'seccre': 0,
 'shdtyp': 1,
 'singopt': 0,
 'slcthk': 1,
 'sliceType': 2,
 'sngpscl': 0,
 'tofTstampScale': 1.9531249881299395e-11,
 'virtualXtal': 0,
 'xXtalPitch': 4.099999904632568,
 'xtalsPerSide': 32,
 'yrcre': 1969,
 'zXtalPitch': 4.099999904632568}

=== Extended Header ===
{'Dpat_id': '00000087',
 'Dpat_name': 'SERVICE_Anonymous^BUTTON SRC EMISSION',
 'assay_date': 0,
 'assay_time': 0,
 'card_phstate': 0,
 'req_phys': '',
 'route': 1,
 'series_desc': '9O Emission QC',
 'series_uid': '1.3.46.670589.28.2.2.440632742027

This will dump out each header’s fields in human-readable form. You can extend each function to cover more of the fields shown in the MATLAB files by following the same pattern of seek, struct.unpack, and assigning into the dict.



## Note: Convert 'tofTstampScale':
from 1.9531249881299395e-11 seconds to 19.531249881 picoseconds

My additional Prompt:

## Can we also print values for c_h_w_b the Coincidence Half Window Bins at 0x07D0 of Extended Main Header?


For two events to be considered by the system as a coincidence event their timestamps must differ
by less than this number of timing bins.
Added for Veyron.
Note: To convert from timing bins to an actual time value in picoseconds, use the tofTstampScale
field, which is the time in picoseconds per timing bin




o4:

It seems the user is looking for specifics on how to print values from an extended header, such as c_h_w_b and other fields like coil_type and pet_software_version. We’ve previously provided code to parse the extended header starting at offset 1024, but it didn’t go through all the necessary fields. Now, they’ve given me a better memory map to parse. The next step is to seek to the right offsets and read the values, especially focusing on c_h_w_b, which is an unsigned short at offset 0x07D0.

It looks like the user is asking about how to work with memory offsets in the extended header, which starts at byte 1024. I need to calculate the absolute file offsets based on the relative ones in the page 4 continuation of the memory map. The extended header's starting point is at 1024, and page 4 offsets like 0x0700 need to be added. I realize the next step will be to adjust the code: I'll use f.seek(1024 + 0x07D0) to correctly access the c_h_w_b field at that location.




In [ ]:
import struct

# Path to your list file
filepath = '3OCLK_2bd6.list'

with open(filepath, 'rb') as f:
    # Extended header base offset
    ext_base = 1024

    # Offset of c_h_w_b within extended header: 0x07D0
    chwb_offset = ext_base + 0x07D0

    # Seek and read one big‑endian uint16
    f.seek(chwb_offset, 0)
    c_h_w_b = struct.unpack('>H', f.read(2))[0]

print(f"Coincidence half-window bins (c_h_w_b) at 0x07D0: {c_h_w_b}")



Coincidence half-window bins (c_h_w_b) at 0x07D0: 119


o4:

That snippet reads the 2-byte unsigned c_h_w_b field at offset 0x07D0 in the extended header and prints it.

If you prefer to integrate this into your read_extended_header function, just add:


    # After all your other reads in read_extended_header(...)
    # Jump to c_h_w_b
    f.seek(1024 + 0x07D0)
    hdr['c_h_w_b'] = struct.unpack('>H', f.read(2))[0]

Then when you call:

ex_hdr = read_extended_header('3OCLK_2bd6.list', main_hdr['file_fmt'])
print("c_h_w_b:", ex_hdr['c_h_w_b'])



you’ll see the scanner’s configured coincidence half-window bins value.






## Note:  
Coincidence half-window bins (c_h_w_b) at 0x07D0: 119 x
'tofTstampScale': 1.9531249881299395e-11 =  19.5 picoseconds =
2,320 picoseconds max allowed tof value to be a coincidence event.

For 676 FOV, the time it takes for light to travel the max 676 mm is approx. 2,260 picoseconds!

The event tof value is in bins, so for our button source taped close to bore cover diameter, the approx. max tof value is 108 to 118 this x 19.5 picoseconds is 2,106 to 2,301 picoseconds.

## Also Note:
A minimum tof difference of 0 would be a normal distribution around 19.5 picoseconds wide.



## Take a brief look at 576 FOV list data:


In [ ]:
import struct

# Path to your list file
#filepath = '3OCLK_2bd6.list'
filepath = '500K_37f3.list'

with open(filepath, 'rb') as f:
    # Extended header base offset
    ext_base = 1024

    # Offset of c_h_w_b within extended header: 0x07D0
    chwb_offset = ext_base + 0x07D0

    # Seek and read one big‑endian uint16
    f.seek(chwb_offset, 0)
    c_h_w_b = struct.unpack('>H', f.read(2))[0]

print(f"Coincidence half-window bins (c_h_w_b) at 0x07D0: {c_h_w_b}")


Coincidence half-window bins (c_h_w_b) at 0x07D0: 103


For 576 FOV time for light to travel full LOR length = 1,920 picoseconds.
listPrint tof max approx. 98 to 103  = 1,911 to 2,008 picoseconds.

